In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install textstat --quiet
!pip install bert-score --quiet
!pip install openai --quiet
!pip install nltk --quiet

import nltk
nltk.download("punkt")




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.9 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install textstat

In [4]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

print("Clave cargada:", os.environ["OPENAI_API_KEY"][:6] + "*****")


Clave cargada: sk-pro*****


In [5]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

#from bert_score import score as bert_score
from nltk.tokenize import sent_tokenize

from openai import OpenAI
client = OpenAI()

import torch
import textstat


In [6]:
from bert_score import BERTScorer ### mas rapida

# Cargar SOLO UNA VEZ
scorer = BERTScorer(lang="en", model_type="roberta-large", device="cuda")

def bert_factuality(text, abstract):
    P, R, F = scorer.score([text], [abstract])
    return float(F[0])

import time

import time

def alignscore(text, reference, retries=5):

    prompt = f"Rate similarity 0-1. A: {reference} B: {text}"

    for attempt in range(1, retries+1):
        try:
            resp = client.responses.create(
                model="gpt-4o-mini",
                input=prompt,
                max_output_tokens=16

            )
            score = float(resp.output_text.strip())
            return max(0.0, min(1.0, score))
        except Exception as e:
            print(f"⚠️ AlignScore error (intent {attempt}/{retries}):", e)
            time.sleep(1.2 + attempt * 0.3)  # backoff

    return 0.0     # fallback si no pudo




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import numpy as np
import torch
import torch.nn.functional as F

def alignscore_emb(text, reference):
    """
    Alignment semántico estable usando embeddings
    Retorna un valor de 0 a 1.
    """

    try:
        emb = client.embeddings.create(
            model="text-embedding-3-large",
            input=[text, reference]
        )

        v1 = np.array(emb.data[0].embedding, dtype=np.float32)
        v2 = np.array(emb.data[1].embedding, dtype=np.float32)

        # Similaridad coseno normalizada a [0, 1]
        score = float((1 + np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))) / 2)

        return max(0, min(1, score))

    except Exception as e:
        print("⚠️ Error en AlignScore EMB:", e)
        return 0.0


In [8]:
!pip install transformers accelerate

In [9]:
import torch
import math
from transformers import AutoTokenizer, AutoModelForCausalLM

# Modelo ligero recomendado
ppl_model = "EleutherAI/gpt-neo-125M"

device = "cuda" if torch.cuda.is_available() else "cpu"

tok = AutoTokenizer.from_pretrained(ppl_model)
# MUY IMPORTANTE
tok.pad_token = tok.eos_token
tok.padding_side = "left"

mdl = AutoModelForCausalLM.from_pretrained(ppl_model).to(device)
mdl.eval()

def perplexity(text, max_length=512):
    """
    Calcula perplexity PARA UNA SOLA FRASE/TEXTO.
    """
    if isinstance(text, list):
        raise ValueError("❌ perplexity(text) recibe SOLO un string, no una lista.")

    enc = tok(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length,
    ).to(device)

    with torch.no_grad():
        loss = mdl(**enc, labels=enc["input_ids"]).loss

    return float(torch.exp(loss))


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [10]:
import textstat

def readability_score(text):
    try:
        grade = textstat.flesch_kincaid_grade(text)
        return float(grade)
    except Exception:
        # algo tipo "nivel bachillerato" como fallback
        return 12.0


In [11]:
import random

def coherence_score_batch(text):
    sentences = sent_tokenize(text)
    if len(sentences) < 2:
        return 0.5

    # pares adyacentes
    refs = sentences[:-1]
    hyps = sentences[1:]

    # BERTScorer hace batch automático
    _, _, F = scorer.score(hyps, refs)

    return float(F.mean())



In [12]:
def conciseness(text, reference):
    len_t = len(text.split())
    len_r = len(reference.split())
    if len_r == 0:
        return 0.0
    ratio = len_t / len_r
    return max(0, 1 - abs(ratio - 1))


In [13]:
def normalize_readability(fkgl):
    fkgl = max(0, min(fkgl, 20))  # recorte higiénico
    return 1 - fkgl / 20          # 0 = horrible, 1 = perfecto


In [14]:
def final_meta_score(bert_f1, align, read, coh, conc, ppl):

    # Transformaciones necesarias
    fluency = 1 / (1 + ppl)           # ya la tenías
    read_norm = normalize_readability(read)  # 🔥 ahora sí

    return (
        0.30 * bert_f1 +
        0.30 * align +
        0.10 * read_norm +   # ✔ reemplaza read → read_norm
        0.15 * coh +
        0.10 * conc
        #0.05 * fluency
    )



In [15]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [16]:
# =======================================
# BASELINE HUMANO (N ajustable)
# =======================================

import numpy as np
import pandas as pd
from tqdm import tqdm

# ---------- CONFIGURACIÓN ----------
HUMAN_CSV = "/content/drive/MyDrive/cochrane_2000_train.csv"

# Cambia este valor a 1000 o 3000:
N_HUMAN = 10   # ⚠️ Pruébalo con 1000 y luego con 3000

# ---------- CARGA ----------
df_human = pd.read_csv(HUMAN_CSV)
df_human = df_human.dropna(subset=["pls", "non_pls"])

print(f"📚 Total PLS humanos disponibles: {len(df_human)}")

# Número real a muestrear
N = min(N_HUMAN, len(df_human))
sample = df_human.sample(N, random_state=42)

print(f"📌 Muestreando {N} PLS humanos para baseline...")




# ---------- BUCLE DE MÉTRICAS (sin perplexity aún) ----------
baseline_results = []

text_list = []   # Para batch perplexity
for _, row in tqdm(sample.iterrows(), total=N, desc="📊 Analizando métricas"):

    pls_text = row["pls"]
    abstract_text = row["non_pls"]

    bert_f1 = bert_factuality(pls_text, abstract_text)
    align = alignscore(pls_text, abstract_text)
    read = readability_score(pls_text)
    coh = coherence_score_batch(pls_text)   # ✔ rápido
    conc = conciseness(pls_text, abstract_text)
    ppl = perplexity(pls_text)              # ← 🔥 aquí va ahora

    meta = final_meta_score(
        bert_f1, align, read, coh, conc, ppl
    )

    baseline_results.append({
        "pls": pls_text,
        "bert_factuality": bert_f1,
        "alignscore": align,
        "readability": read,
        "coherence": coh,
        "conciseness": conc,
        "perplexity": ppl,
        "final_meta_score": meta
    })

df_human_eval = pd.DataFrame(baseline_results)

# ---------- GUARDAR ----------

out_path = f"/content/drive/MyDrive/human_baseline_{N}.csv"
df_human_eval.to_csv(out_path, index=False)

print("\n===========================")
print(f"📁 Baseline guardado en: {out_path}")
print("===========================\n")

# ---------- ESTADÍSTICAS ----------
df_human_eval.describe()


📚 Total PLS humanos disponibles: 2000
📌 Muestreando 10 PLS humanos para baseline...


📊 Analizando métricas:   0%|          | 0/10 [00:00<?, ?it/s]

⚠️ AlignScore error (intent 1/5): could not convert string to float: 'I would rate the similarity between A and B as **0.9**.'
⚠️ AlignScore error (intent 2/5): could not convert string to float: 'I would rate the similarity between text A and text B at **0.9'
⚠️ AlignScore error (intent 3/5): could not convert string to float: 'To rate the similarity between texts A and B on a scale of 0 to'
⚠️ AlignScore error (intent 4/5): could not convert string to float: 'I would rate the similarity between the two texts as **0.9**.'
⚠️ AlignScore error (intent 5/5): could not convert string to float: 'I would rate the similarity between the two texts as **0.9**.'


📊 Analizando métricas:  10%|█         | 1/10 [00:26<03:54, 26.04s/it]

⚠️ AlignScore error (intent 1/5): could not convert string to float: 'I would rate the similarity between sections A and B as **0.9**'
⚠️ AlignScore error (intent 2/5): could not convert string to float: 'The similarity rating between passages A and B is approximately **0.8**.'
⚠️ AlignScore error (intent 3/5): could not convert string to float: 'I would rate the similarity between the two texts at **0.9**.'
⚠️ AlignScore error (intent 4/5): could not convert string to float: "I'd rate the similarity between the two texts as **0.9**.\n\n###"
⚠️ AlignScore error (intent 5/5): could not convert string to float: 'Based on the content provided, I would rate the similarity between Text A and Text'


📊 Analizando métricas:  10%|█         | 1/10 [00:47<07:11, 47.98s/it]


KeyboardInterrupt: 

In [ ]:
def evaluate_model(model_name, df):

    required_cols = ["generated_text", "source", "response"]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"❌ La columna '{col}' NO existe en el CSV.")

    subset = df.sample(50, random_state=42)

    results = []

    for _, row in tqdm(subset.iterrows(), total=len(subset), desc=f"Eval {model_name}"):

        text = row["generated_text"]
        abstract = row["source"]
        response = row["response"]

        # ---- métrica core ----
        bscore = bert_factuality(text, response)
        #ascore = alignscore_emb(text, abstract)
        ascore = alignscore(text, abstract)
        read = readability_score(text)
        coh = coherence_score_batch(text)
        conc = conciseness(text, abstract)
        ppl = perplexity(text)

        meta = final_meta_score(bscore, ascore, read, coh, conc, ppl)

        results.append({
            "bert": bscore,
            "align": ascore,
            "read": read,
            "coh": coh,
            "conc": conc,
            "ppl": ppl,
            "meta": meta
        })

    return pd.DataFrame(results)


In [ ]:
import pandas as pd

BASE_PATH = "/content/drive/MyDrive"

# 🔧 Definir solo el nombre del archivo (sin .csv)
CSV_NAME ="pls_results_PLS_v6_L4_2_epochs_CE_KL_PROMPT1_2000_instruct_Low_v2-check380"  ### <---

# 🔧 Construir el path completo (auto .csv)
MODEL_CSV = f"{BASE_PATH}/{CSV_NAME}.csv"

# 📥 Cargar
df_models = pd.read_csv(MODEL_CSV)

print("📄 Cargado desde:", MODEL_CSV)
print(df_models.head())


📄 Cargado desde: /content/drive/MyDrive/pls_results_PLS_v6_L4_2_epochs_CE_KL_PROMPT1_2000_instruct_Low_v2-check380.csv
                                              source  \
0  Background\nThis updated Cochrane Review of re...   
1  Background\nIn many countries emergency depart...   
2  Background\nPsychotherapy is regarded as the f...   
3  Background\nSpina bifida is a fetal neural tub...   
4  We included 12 trials in the original review (...   

                                            response  \
0  Reminiscence therapy for dementia\nReview ques...   
1  Primary care professionals providing non‐urgen...   
2  Psychological therapies for borderline persona...   
3  Spina bifida repair and infant and maternal he...   
4  Transcutaneous Electrical Nerve Stimulation (T...   

                               source_for_generation  \
0  Background This updated Cochrane Review of rem...   
1  Background In many countries emergency departm...   
2  Background Psychotherapy is regarded

In [ ]:
models = [CSV_NAME]   # ← una lista con un solo modelo

global_results = {}

for m in models:
    print(f"\n\n============================")
    print(f" Evaluando modelo: {m}")
    print(f"============================")

    stats = evaluate_model(m, df_models)
    global_results[m] = stats

    mean_meta = stats["meta"].mean()
    mean_ppl = stats["ppl"].mean()

    print("\n 📊 Media Meta Score:", mean_meta)
    print("📉 Media Perplexity:", mean_ppl)

    stats.to_csv(f"/content/drive/MyDrive/eval_{m}.csv")
    print(f"📁 Guardado eval_{m}.csv")




 Evaluando modelo: pls_results_PLS_v6_L4_2_epochs_CE_KL_PROMPT1_2000_instruct_Low_v2-check380


Eval pls_results_PLS_v6_L4_2_epochs_CE_KL_PROMPT1_2000_instruct_Low_v2-check380: 100%|██████████| 50/50 [00:43<00:00,  1.14it/s]


 📊 Media Meta Score: 0.7389183510039746
📉 Media Perplexity: 10.671444854736329
📁 Guardado eval_pls_results_PLS_v6_L4_2_epochs_CE_KL_PROMPT1_2000_instruct_Low_v2-check380.csv
